# Multi-Cycle and Parameters

In this notebook, we cover the ability to simulation over a larger range of conditions inclusive of multi-cycling via the 'experimental' function call. First, we install and import PyBaMM.

In [14]:
%pip install pybamm -q    # install PyBaMM if it is not installed
import pybamm
import matplotlib.pyplot as plt


[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


An experimental defines conditional progression of the battery, and is covers common protocols like a CC-CV operation. In the following definition, a cycle is defined by a tuple of operating instructions. In this case, the experiment consists of a cycle of constant current C/10 discharge, a one hour rest, a constant current (1 A) constant voltage (4.2 V) and another one hour rest, all of it repeated three times (notice the * 3).

In [10]:
experiment = pybamm.Experiment(
    [
        ("Discharge at C/10 for 10 hours or until 3.0 V",
        "Rest for 1 hour",
        "Charge at 1 A until 4.1 V",
        "Hold at 4.1 V until 50 mA",
        "Rest for 1 hour"),
    ] * 3
)


Next, we re-introduce the DFN model,

In [11]:
model = pybamm.lithium_ion.DFN()

and create our simulation, passing our experiment using a keyword argument

In [12]:
sim = pybamm.Simulation(model, experiment=experiment)

We then solve and plot the solution

In [19]:
sim.solve()
sim.plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=38.90905254448877, step=0.38909052544488776)…

As we have seen, experiments allow us to define complex simulations using a very simple syntax. The instructions can be of the form "(Dis)charge at x A/C/W", "Rest", or "Hold at x V". The running time should be a time in seconds, minutes or hours, e.g. "10 seconds", "3 minutes" or "1 hour". The stopping conditions should be a circuit state, e.g. "1 A", "C/50" or "3 V". 

Some examples of experiment instructions are:
```python
    "Discharge at 1C for 0.5 hours",
    "Discharge at C/20 for 0.5 hours",
    "Charge at 0.5 C for 45 minutes",
    "Discharge at 1 A for 90 seconds",
    "Charge at 200mA for 45 minutes (1 minute period)",
    "Discharge at 1 W for 0.5 hours",
    "Charge at 200 mW for 45 minutes",
    "Rest for 10 minutes (5 minute period)",
    "Hold at 1 V for 20 seconds",
    "Charge at 1 C until 4.1V",
    "Hold at 4.1 V until 50 mA",
    "Hold at 3V until C/50",
```

Optionally, each instruction can contain at the end the expression "(x minute period)" in which the period at which to record the simulation outputs during that instruction. To change the period for the whole experiment we can pass it as a keyword argument in the experiment.

Additionally, we can use the operators `+` and `*` on lists in order to combine and repeat cycles:

In [6]:
[("Discharge at 1C for 0.5 hours", "Discharge at C/20 for 0.5 hours")] * 3 + [("Charge at 0.5 C for 45 minutes",)]

[('Discharge at 1C for 0.5 hours', 'Discharge at C/20 for 0.5 hours'),
 ('Discharge at 1C for 0.5 hours', 'Discharge at C/20 for 0.5 hours'),
 ('Discharge at 1C for 0.5 hours', 'Discharge at C/20 for 0.5 hours'),
 ('Charge at 0.5 C for 45 minutes',)]

# Modifying parameter values

Previously, we've ran all of our models with the default parameter values; However, PyBaMM also allows you to tweak these settings for your application. In this tutorial, we will see how to change the parameters in PyBaMM. They can be viewed as,

In [24]:
model.default_parameter_values

{'1 + dlnf/dlnc': 1.0,
 'Ambient temperature [K]': 298.15,
 'Bulk solvent concentration [mol.m-3]': 2636.0,
 'Cation transference number': 0.4,
 'Cell cooling surface area [m2]': 0.0569,
 'Cell volume [m3]': 7.8e-06,
 'Contact resistance [Ohm]': 0,
 'Current function [A]': 0.680616,
 'EC diffusivity [m2.s-1]': 2e-18,
 'EC initial concentration in electrolyte [mol.m-3]': 4541.0,
 'Edge heat transfer coefficient [W.m-2.K-1]': 0.3,
 'Electrode height [m]': 0.137,
 'Electrode width [m]': 0.207,
 'Electrolyte conductivity [S.m-1]': <function electrolyte_conductivity_Capiglia1999 at 0x12b54c940>,
 'Electrolyte diffusivity [m2.s-1]': <function electrolyte_diffusivity_Capiglia1999 at 0x12b54c8b0>,
 'Initial concentration in electrolyte [mol.m-3]': 1000.0,
 'Initial concentration in negative electrode [mol.m-3]': 19986.609595075,
 'Initial concentration in positive electrode [mol.m-3]': 30730.7554385565,
 'Initial inner SEI thickness [m]': 2.5e-09,
 'Initial outer SEI thickness [m]': 2.5e-09,
 

Note, the default parameter set is "Marquis2019".

## Changing the parameter set

PyBaMM has a number of in-built parameter sets (check the list [here](https://pybamm.readthedocs.io/en/latest/source/api/parameters/parameter_sets.html)), which can be selected via,

In [26]:
parameter_values = pybamm.ParameterValues("Chen2020")

Likewise to above, we can display the parameters and corresponding values stored in the dictionary,

In [27]:
parameter_values

{'1 + dlnf/dlnc': 1.0,
 'Ambient temperature [K]': 298.15,
 'Bulk solvent concentration [mol.m-3]': 2636.0,
 'Cation transference number': 0.2594,
 'Cell cooling surface area [m2]': 0.00531,
 'Cell thermal expansion coefficient [m.K-1]': 1.1e-06,
 'Cell volume [m3]': 2.42e-05,
 'Contact resistance [Ohm]': 0,
 'Current function [A]': 5.0,
 'EC diffusivity [m2.s-1]': 2e-18,
 'EC initial concentration in electrolyte [mol.m-3]': 4541.0,
 'Electrode height [m]': 0.065,
 'Electrode width [m]': 1.58,
 'Electrolyte conductivity [S.m-1]': <function electrolyte_conductivity_Nyman2008 at 0x1330ad160>,
 'Electrolyte diffusivity [m2.s-1]': <function electrolyte_diffusivity_Nyman2008 at 0x1330ad790>,
 'Initial concentration in electrolyte [mol.m-3]': 1000.0,
 'Initial concentration in negative electrode [mol.m-3]': 29866.0,
 'Initial concentration in positive electrode [mol.m-3]': 17038.0,
 'Initial inner SEI thickness [m]': 2.5e-09,
 'Initial outer SEI thickness [m]': 2.5e-09,
 'Initial temperature

or we can search for a particular parameter

In [28]:
parameter_values.search("electrolyte")

EC initial concentration in electrolyte [mol.m-3]	4541.0
Electrolyte conductivity [S.m-1]	<function electrolyte_conductivity_Nyman2008 at 0x1330ad160>
Electrolyte diffusivity [m2.s-1]	<function electrolyte_diffusivity_Nyman2008 at 0x1330ad790>
Initial concentration in electrolyte [mol.m-3]	1000.0
Negative electrode Bruggeman coefficient (electrolyte)	1.5
Positive electrode Bruggeman coefficient (electrolyte)	1.5
Separator Bruggeman coefficient (electrolyte)	1.5
Typical electrolyte concentration [mol.m-3]	1000.0


To run a simulation with this parameter set, we can proceed as usual but passing the parameters as a keyword argument

In [30]:
sim = pybamm.Simulation(model, parameter_values=parameter_values, experiment=experiment)
sim.solve()
sim.plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=52.691215978330206, step=0.5269121597833021)…

More details on each subset can be found [here](https://github.com/pybamm-team/PyBaMM/tree/develop/pybamm/input/parameters).

## Change individual parameters

We often want to quickly change a small number of parameter values to investigate how the behaviour or the battery changes. In such cases, we can change parameter values without having to leave the notebook or script you are working in.

We will modify the contact resistance variable to 10 mOhm,

In [32]:
parameter_values["Contact resistance [Ohm]"] = 0.01

Now we just need to run the simulation with the new parameter values

In [33]:
sim = pybamm.Simulation(model, experiment=experiment, parameter_values=parameter_values)
sim.solve()
sim.plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=52.691318832995464, step=0.5269131883299546)…

# Simulating a  Drive-Cycle

You can implement drive cycles importing the dataset and creating an interpolant to pass as the current function.

In [45]:
import pandas as pd    # needed to read the csv data file
import numpy as np

drive_cycle_power = pd.read_csv("Data/WLTP_M50_M3.csv", comment="#", header=None).to_numpy()

We can then include the previous experiment definition as,

In [46]:
# Create interpolant
timescale = parameter_values.evaluate(model.timescale)
power_interpolant = pybamm.Interpolant(drive_cycle_power[:, 0], drive_cycle_power[:, 1], timescale * pybamm.t)

# Set drive cycle power function
parameter_values.update({"Power function [W]": power_interpolant},check_already_exists=False)
t_eval = np.linspace(0, 1800, 7201)

Note that your drive cycle data can be stored anywhere, you just need to pass the path of the file. Then, again, the model can be solved as usual but notice that now, if `t_eval` is not specified, the solver will take the time points from the data set.

In [48]:
model = pybamm.lithium_ion.SPMe({"operating mode": "power"})
sim = pybamm.Simulation(model, parameter_values=parameter_values)
sim.solve(t_eval)
sim.plot(["Current [A]", "Terminal voltage [V]"])

interactive(children=(FloatSlider(value=0.0, description='t', max=1800.0, step=18.0), Output()), _dom_classes=…

Now, we can set the initial SOC for the drive-cycle by passing it via keyword arguments, 

In [49]:
sim.solve(t_eval,initial_soc=0.717)
sim.plot(["Current [A]", "Terminal voltage [V]"])

interactive(children=(FloatSlider(value=0.0, description='t', max=1800.0, step=18.0), Output()), _dom_classes=…

Reusing the model comparison from the previous notebook, we can create a list for comparing models on the WLTP drive-cycle as:

In [52]:
models = [
    pybamm.lithium_ion.SPM({"operating mode": "power"}),
    pybamm.lithium_ion.SPMe({"operating mode": "power"}),
    pybamm.lithium_ion.DFN({"operating mode": "power"}),
]

Note that we update the model definition for the power operation mode. Next, we loop through the formed list and append the solutions to the `sim` list.

In [53]:
sims = []
for model in models:
    sim = pybamm.Simulation(model, parameter_values=parameter_values)
    sim.solve(t_eval)
    sims.append(sim)

Plotting the comparison as,

In [54]:
pybamm.dynamic_plot(sims, time_unit="seconds")

interactive(children=(FloatSlider(value=0.0, description='t', max=1800.0, step=18.0), Output()), _dom_classes=…

# Accessing and saving simulation outputs

We can now access the solved variables directly to visualise or create our own plots. We first extract the solution object:

In [ ]:
solution = sims[0].solution

and now we can create a post-processed variable,

In [ ]:
t = solution["Time [s]"]
V = solution["Terminal voltage [V]"]

One option is to visualise the data set returned by the solver directly

In [ ]:
V.entries

array([3.77047806, 3.75305182, 3.74567027, 3.74038822, 3.73581196,
       3.73153391, 3.72742393, 3.72343929, 3.71956623, 3.71580184,
       3.71214621, 3.7086004 , 3.70516561, 3.70184253, 3.69863121,
       3.69553118, 3.69254137, 3.68966018, 3.68688562, 3.68421526,
       3.68164637, 3.67917591, 3.6768006 , 3.67451688, 3.67232094,
       3.67020869, 3.66817572, 3.66621717, 3.66432762, 3.6625009 ,
       3.66072974, 3.65900536, 3.65731692, 3.65565066, 3.65398895,
       3.65230898, 3.65058135, 3.6487688 , 3.64682546, 3.64469798,
       3.64232968, 3.63966973, 3.63668796, 3.63339303, 3.62984711,
       3.62616692, 3.6225045 , 3.61901241, 3.61580868, 3.6129572 ,
       3.61046847, 3.60831405, 3.60644483, 3.60480596, 3.60334607,
       3.60202167, 3.60079822, 3.5996495 , 3.59855637, 3.59750531,
       3.59648723, 3.59549638, 3.59452954, 3.59358541, 3.59266405,
       3.59176646, 3.59089417, 3.59004885, 3.58923192, 3.58844407,
       3.58768477, 3.58695179, 3.58624057, 3.58554372, 3.58485

which correspond to the data at the times

In [ ]:
t.entries

array([   0.        ,   36.36363636,   72.72727273,  109.09090909,
        145.45454545,  181.81818182,  218.18181818,  254.54545455,
        290.90909091,  327.27272727,  363.63636364,  400.        ,
        436.36363636,  472.72727273,  509.09090909,  545.45454545,
        581.81818182,  618.18181818,  654.54545455,  690.90909091,
        727.27272727,  763.63636364,  800.        ,  836.36363636,
        872.72727273,  909.09090909,  945.45454545,  981.81818182,
       1018.18181818, 1054.54545455, 1090.90909091, 1127.27272727,
       1163.63636364, 1200.        , 1236.36363636, 1272.72727273,
       1309.09090909, 1345.45454545, 1381.81818182, 1418.18181818,
       1454.54545455, 1490.90909091, 1527.27272727, 1563.63636364,
       1600.        , 1636.36363636, 1672.72727273, 1709.09090909,
       1745.45454545, 1781.81818182, 1818.18181818, 1854.54545455,
       1890.90909091, 1927.27272727, 1963.63636364, 2000.        ,
       2036.36363636, 2072.72727273, 2109.09090909, 2145.45454

In addition, post-processed variables can be called at any time (by interpolation)

In [ ]:
V([200, 400, 780, 1236])  # times in seconds

array([3.72947892, 3.7086004 , 3.67810702, 3.65400557])

## Saving the simulation and output data

In some cases simulations might take a long time to run so it is advisable to save in your computer so it can be analysed later without re-running the simulation. You can save the whole simulation doing:

In [ ]:
sims[1].save("SPMe.pkl")

If you now check the root directory of your notebooks you will notice that a new file called `"SPMe.pkl"` has appeared. We can load the stored simulation doing

In [ ]:
sim2 = pybamm.load("SPMe.pkl")

which allows the same manipulation as the original simulation would allow

In [ ]:
sim2.plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

Alternatively, we can just save the solution of the simulation in a similar way

In [ ]:
sol = sim.solution
sol.save("SPMe_sol.pkl")

and load it in a similar way too

In [ ]:
sol2 = pybamm.load("SPMe_sol.pkl")
pybamm.dynamic_plot(sol2)

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

Another option is to just save the data for some variables

In [ ]:
sol.save_data("sol_data.pkl", ["Current [A]", "Terminal voltage [V]"])

or save in csv or mat format

In [ ]:
sol.save_data("sol_data.csv", ["Current [A]", "Terminal voltage [V]"], to_format="csv")
# matlab needs names without spaces
sol.save_data("sol_data.mat", ["Current [A]", "Terminal voltage [V]"], to_format="matlab",
              short_names={"Current [A]": "I", "Terminal voltage [V]": "V"})

If you are running on your local, removing the generated files can be completed through the below code block.

In [ ]:
import os
os.remove("SPMe.pkl")
os.remove("SPMe_sol.pkl")
os.remove("sol_data.pkl")
os.remove("sol_data.csv")
os.remove("sol_data.mat")

## References

The relevant papers for this notebook are:

In [7]:
pybamm.print_citations()

[1] Joel A. E. Andersson, Joris Gillis, Greg Horn, James B. Rawlings, and Moritz Diehl. CasADi – A software framework for nonlinear optimization and optimal control. Mathematical Programming Computation, 11(1):1–36, 2019. doi:10.1007/s12532-018-0139-4.
[2] Marc Doyle, Thomas F. Fuller, and John Newman. Modeling of galvanostatic charge and discharge of the lithium/polymer/insertion cell. Journal of the Electrochemical society, 140(6):1526–1533, 1993. doi:10.1149/1.2221597.
[3] Charles R. Harris, K. Jarrod Millman, Stéfan J. van der Walt, Ralf Gommers, Pauli Virtanen, David Cournapeau, Eric Wieser, Julian Taylor, Sebastian Berg, Nathaniel J. Smith, and others. Array programming with NumPy. Nature, 585(7825):357–362, 2020. doi:10.1038/s41586-020-2649-2.
[4] Scott G. Marquis, Valentin Sulzer, Robert Timms, Colin P. Please, and S. Jon Chapman. An asymptotic derivation of a single particle model with electrolyte. Journal of The Electrochemical Society, 166(15):A3693–A3706, 2019. doi:10.1149/